In [3]:
import pandas as pd
import pyvista as pv
from glob import glob
import json
import os
import numpy as np
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
pv.start_xvfb()

In [5]:
tex_info = pd.read_json("/home/aleksandr/mount-folder/3D-FRONT-texture/texture_info.json")

In [12]:
tex_room = pv.read_texture("/home/aleksandr/mount-folder/3D-FRONT-texture/273a7787-72f5-4642-b333-03d06526035b/texture.png")

In [43]:
! ls /home/aleksandr/mount-folder/3D-FUTURE-model-part1 

In [4]:
f = open("existing_models.txt")
existing_model_ids = set(f.read().split("\n"))

In [5]:
model_info = pd.read_json("/home/aleksandr/mount-folder/3D-FUTURE-model-part1/model_info.json")
model_info

,model_id,super-category,category,style,theme,material
0,f89da2db-ad8c-4582-b186-ed2a46f3cb15,Sofa,armchair,Minimalist,Gold Foil,Rough Cloth
1,5de45849-7d1b-4378-82da-ed183b7ecc37,Chair,Lounge Chair / Cafe Chair / Office Chair,Ming Qing,Floral,Rough Cloth
2,1ef668a9-12e0-447b-9bbc-8ae484ba8c58,Lighting,Pendant Lamp,Modern,None,None
3,0d3e3b3c-3f1a-47ee-8566-1052cb8635b6,Cabinet/Shelf/Desk,Coffee Table,Modern,Texture Mark,Wood
4,16febe29-52d9-4833-a695-b09446e65512,Cabinet/Shelf/Desk,Corner/Side Table,Modern,Cartoon,Composite Board
...,...,...,...,...,...,...
16558,328918d2-1121-4f2e-bb63-fb1ecadc687a,Others,None,Modern,None,None
16559,5e5c01c1-e9a0-4eff-a059-0c9ea5afae9c,Lighting,Ceiling Lamp,Modern,None,None
16560,908e38ac-35ca-4aa2-ad9f-8402fc7ae8ed,Others,None,Modern,None,None
16561,966e026d-c284-438e-99a1-2baba6379d3f,Others,None,Modern,None,None


In [6]:
# model_info.groupby("super-category")["category"].unique().explode()

In [6]:
n = 3
dir = "/home/aleksandr/mount-folder/3D-FUTURE-model-part1"
# model_ids = existing_model_ids & set(model_info["model_id"])
model_ids = existing_model_ids & set(model_info[model_info["category"] == "King-size Bed"]["model_id"])
model_ids = list(model_ids)
models = []
for name in model_ids[:n]:
    f = os.path.join(dir, name)
    models.append({
        "model": pv.read(os.path.join(f, "normalized_model.obj")),
        "texture": pv.read_texture(os.path.join(f, "texture.png"))
    })
# m = pv.read("/home/aleksandr/mount-folder/3D-FUTURE-model-part1/ffde330f-dce0-4215-bfe6-203fe615b609/raw_model.obj")
# norm_m = pv.read("/home/aleksandr/mount-folder/3D-FUTURE-model-part1/ffde330f-dce0-4215-bfe6-203fe615b609/normalized_model.obj")
# tex = pv.read_texture("/home/aleksandr/mount-folder/3D-FUTURE-model-part1/ffde330f-dce0-4215-bfe6-203fe615b609/texture.png")

In [7]:
def crop_object(image_array):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)

    gray = np.where(gray==255, 0, gray)

    # # Find contours in the binary mask
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # Find the largest contour (assumed to be the object)
    largest_contour = max(contours, key=cv2.contourArea).squeeze()

    bottom_right = largest_contour.max(0)
    top_left = largest_contour.min(0)

    # TODO: add boundaries checker
    cropped_image_array = image_array[top_left[1]: bottom_right[1] + 1,
                                      top_left[0]: bottom_right[0] + 1, :]
    return cropped_image_array

def create_views(mesh, texture, camera_distance, positions):
    model_views = []
    for position in positions:
        pl = pv.Plotter()
        pl.camera_position = position
        no_transp_texture = pv.numpy_to_texture(texture.to_array()[..., :3])
        pl.add_mesh(mesh, texture=no_transp_texture)
        model_views.append(crop_object(pl.screenshot(return_img=True)))
    return model_views

In [8]:
views = create_views(models[0]["model"], models[0]["texture"])

In [125]:
# # axes = pv.Axes(show_actor=True, actor_scale=1.0)
# camera_distance=5

# # front_view_camera_position = np.array([(0, -camera_distance, 0), (0, 0, 0), (0, 0, 1)])
# # rotated_camera_position = front_view_camera_position.dot(np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 1]]))
# # plotter.camera_position = rotated_camera_position
# positions = [
#     [(0, camera_distance, 0), (0, 0, 0), (0, 0, -1)],
#     [(0, -camera_distance, 0), (0, 0, 0), (0, 0, -1)],
#     [(0, 0, camera_distance), (0, 0, 0), (0, 1, 0)],
#     [(-camera_distance, 0, 0), (0, 0, 0), (0, 1, 0)],
#     [(camera_distance, 0, 0), (0, 0, 0), (0, 1, 0)],
#     [(0, 0, -camera_distance), (0, 0, 0), (0, 1, 0)],
# ]
# views = []
# for m in models:
# #     pl.camera_position = [(3.2257270137110754, 3.640847078923325, 3.2256104866305395),
# #  (0.0, 0.41512006521224976, -0.00011652708053588867),
# #  (0.0, 0.0, 1.0)]
#     # pl.camera.zoom(0.3)
#     # pl.add_actor(axes.actor)
#     one_model_views = []
#     for position in positions:
#         pl = pv.Plotter()
#         pl.camera_position = position
#         pl.add_mesh(m["model"], texture=m["texture"])
#         # pl.add_texture(texture=m["texture"])
#         one_model_views.append(crop_object(pl.screenshot(return_img=True)))
#     views.append(one_model_views)

In [11]:


class FeatureExtractor:
    def __init__(self, model_name='resnet50', feature_layer=None):
        self.model_name = model_name
        self.feature_layer = feature_layer
        
        # Load the pretrained model
        self.model = self._load_pretrained_model()
        self.model.eval()
        
        # Define the transformations
        self.transform = self._get_transform()
    
    def _load_pretrained_model(self):
        # Load the pretrained model from torchvision
        if self.model_name == 'resnet50':
            model = models.resnet50(pretrained=True)
        elif self.model_name == 'vgg16':
            model = models.vgg16(pretrained=True)
        else:
            raise ValueError('Invalid model_name. Supported models: resnet50, vgg16')
        
        # Replace the last fully connected layer with an identity layer
        if self.feature_layer is not None:
            model.fc = torch.nn.Identity()
        
        # Move the model to GPU if available
        if torch.cuda.is_available():
            model = model.cuda()
        
        return model
    
    def _get_transform(self):
        # Define the transformations based on the chosen model
        if self.model_name == 'resnet50':
            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        elif self.model_name == 'vgg16':
            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        else:
            raise ValueError('Invalid model_name. Supported models: resnet50, vgg16')
        
        return transform
    
    def extract_features(self, images):
        _images = images if isinstance(images, list) else [images]

        # Apply transformations to the image
        transformed_images = []
        for image in _images:
            transformed_images.append(self.transform(image))
        transformed_images = torch.stack(transformed_images)
        
        # Move the image to GPU if available
        if torch.cuda.is_available():
            transformed_images = transformed_images.cuda()
        
        # Forward pass through the model
        with torch.no_grad():
            features = self.model(transformed_images)
        
        # Return the concatenated features
        return torch.cat(features.cpu().squeeze())


In [ ]:
class StyleRanker:
    def __init__(
        self, 
        model_name="resnet50",
        feature_layer=None, 
        positions=None, 
        camera_distance=5
    ):
        self.feature_extractor = FeatureExtractor(model_name=model_name, feature_layer=feature_layer)
        self.cossim = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        self.camera_distance = camera_distance
        if not positions:
            self.positions = [
                [(0, camera_distance, 0), (0, 0, 0), (0, 0, -1)],
                [(0, -camera_distance, 0), (0, 0, 0), (0, 0, -1)],
                [(0, 0, camera_distance), (0, 0, 0), (0, 1, 0)],
                [(-camera_distance, 0, 0), (0, 0, 0), (0, 1, 0)],
                [(camera_distance, 0, 0), (0, 0, 0), (0, 1, 0)],
                [(0, 0, -camera_distance), (0, 0, 0), (0, 1, 0)],
            ]
    
    def _extract_features_from_object(self, object):
            object_views = create_views(
                object["model"], 
                object["texture"], 
                camera_distance=self.camera_distance,
                positions=self.positions)
            return self.feature_extractor.extract_features(object_views)

    def __call__(self, context, targets):
        _targets = targets if isinstance(targets, list) else [targets]
        _context = context if isinstance(context, list) else [context]

        aggregated_features = None
        for context_object in _context:
            features = self._extract_features_from_object(context_object)
            aggregated_features = (
                aggregated_features + features) if aggregated_features else features

        scores = torch.zeros(len(_targets))
        for idx, target in enumerate(_targets):
            features = self._extract_features_from_object(target)
            scores[i] = self.cossim(aggregated_features, features)
        return scores

In [13]:
features.shape

torch.Size([1000])